# T2. dataloader 和 tokenizer 的基本使用

&emsp; 1 &ensp; fastNLP 中的 dataloader

&emsp; &emsp; 1.1 &ensp; databundle 的结构与使用

&emsp; &emsp; 1.2 &ensp; dataloader 的结构与使用

&emsp; 2 &ensp; fastNLP 中的 tokenizer
 
&emsp; &emsp; 2.1 &ensp; 传统 GloVe 词嵌入的加载
 
&emsp; &emsp; 2.2 &ensp; PreTrainedTokenizer 的概念

&emsp; &emsp; 2.3 &ensp; BertTokenizer 的基本使用

&emsp; 3 &ensp; 实例：NG20 数据集的完整加载过程
 
&emsp; &emsp; 3.1 &ensp; 

&emsp; &emsp; 3.2 &ensp; 

## 1. fastNLP 中的 dataloader

### 1.1 databundle 的结构与使用

在`fastNLP 0.8`中，在常用的数据加载模块`DataLoader`和数据集`DataSet`模块之间，还存在

&emsp; 一个中间模块，即 **数据包`DataBundle`模块**，可以从`fastNLP.io`路径中导入该模块

在`fastNLP 0.8`中，**一个`databundle`数据包包含若干`dataset`数据集和`vocabulary`词汇表**

&emsp; 分别存储在`datasets`和`vocabs`两个变量中，所以了解`databundle`数据包之前

&emsp; 需要首先**复习`dataset`数据集和`vocabulary`词汇表**，**下面的一串代码**，**你知道其大概含义吗？**

必要提示：`NG20`，全称[`News Group 20`](http://qwone.com/~jason/20Newsgroups/)，是一个新闻文本分类数据集，包含20个大类以及若干小类

&emsp; 数据集包含训练集`'ng20_train.csv'`和测试集`'ng20_test.csv'`两部分，每条数据

&emsp; 包括`'label'`标签和`'text'`文本两个条目，通过`sample(frac=1)[:10]`随机采样并读取前十条

In [1]:
import pandas as pd

from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import DataBundle

datasets = {}
datasets['train'] = DataSet.from_pandas(pd.read_csv('./data/ng20_train.csv').sample(frac=1)[:10])
datasets['train'].apply_more(lambda ins:{'label': ins['label'].lower().split('.')[0], 
                                         'text': ins['text'].lower().split()},
                             progress_bar='tqdm')
datasets['test'] = DataSet.from_pandas(pd.read_csv('./data/ng20_test.csv').sample(frac=1)[:10])
datasets['test'].apply_more(lambda ins:{'label': ins['label'].lower().split('.')[0], 
                                        'text': ins['text'].lower().split()},
                            progress_bar='tqdm')
print(datasets['train'])

vocabs = {}
vocabs['label'] = Vocabulary().from_dataset(datasets['train'].concat(datasets['test'], inplace=False), field_name='label')
vocabs['text'] = Vocabulary().from_dataset(datasets['train'].concat(datasets['test'], inplace=False), field_name='text')
print(vocabs['label'].word2idx)

Processing:   0%|          | 0/10 [00:00<?, ?it/s]

Processing:   0%|          | 0/10 [00:00<?, ?it/s]

+-------+------------------------------------------+
| label | text                                     |
+-------+------------------------------------------+
| talk  | ['mwilson', 'ncratl', 'atlantaga', 'n... |
| talk  | ['ch981', 'cleveland', 'freenet', 'ed... |
| rec   | ['mbeaving', 'bnr', 'ca', '\\(', 'bea... |
| soc   | ['jayne', 'mmalt', 'guild', 'org', '\... |
| talk  | ['jrutledg', 'cs', 'ulowell', 'edu', ... |
| talk  | ['cramer', 'optilink', 'com', '\\(', ... |
| comp  | ['triton', 'unm', 'edu', '\\(', 'larr... |
| rec   | ['ingres', 'com', '\\(', 'bruce', '\\... |
| comp  | ['ldo', 'waikato', 'ac', 'nz', '\\(',... |
| misc  | ['rebecca', 'rpi', 'edu', '\\(', 'ezr... |
+-------+------------------------------------------+
{'<pad>': 0, '<unk>': 1, 'rec': 2, 'talk': 3, 'comp': 4, 'soc': 5, 'misc': 6, 'sci': 7}



数据集（比如：分开的训练集、验证集和测试集）以及各个field对应的vocabulary。
    该对象一般由fastNLP中各种Loader的load函数生成，可以通过以下的方法获取里面的内容

In [2]:
data_bundle = DataBundle(datasets=datasets, vocabs=vocabs)
print(data_bundle)

In total 2 datasets:
	train has 10 instances.
	test has 10 instances.
In total 2 vocabs:
	label has 8 entries.
	text has 1687 entries.



### 1.2 dataloader 的结构与使用

## 2. fastNLP 中的 tokenizer

### 2.1 传统 GloVe 词嵌入的加载

### 2.2 PreTrainTokenizer 的提出

在`fastNLP 0.8`中，**使用`PreTrainedTokenizer`模块来为数据集中的词语进行词向量的标注**

&emsp; 需要注意的是，`PreTrainedTokenizer`模块的下载和导入**需要确保环境安装了`transformers`模块**

&emsp; 这是因为 `fastNLP 0.8`中`PreTrainedTokenizer`模块的实现基于`Huggingface Transformers`库

**`Huggingface Transformers`是基于一个开源的**，**基于`transformer`模型结构提供的预训练语言库**

&emsp; 包含了多种经典的基于`transformer`的预训练模型，如`BERT`、`BART`、`RoBERTa`、`GPT2`、`CPT`

&emsp; 更多相关内容可以参考`Huggingface Transformers`的[相关论文](https://arxiv.org/pdf/1910.03771.pdf)、[官方文档](https://huggingface.co/transformers/)以及[的代码仓库](https://github.com/huggingface/transformers)

### 2.3 BertTokenizer 的基本使用

在`fastNLP 0.8`中，以`PreTrainedTokenizer`为基类，泛化出多个子类，实现基于`BERT`等模型的标注

&emsp; 本节以`BertTokenizer`模块为例，展示`PreTrainedTokenizer`模块的使用方法与应用实例

**`BertTokenizer`的初始化包括 导入模块和导入数据 两步**，先通过从`fastNLP.transformers.torch`中

&emsp; 导入`BertTokenizer`模块，再通过`from_pretrained`方法指定`tokenizer`参数类型下载

&emsp; 其中，**`'bert-base-uncased'`指定`tokenizer`使用的预训练`BERT`类型**：单词不区分大小写

&emsp; &emsp; **模块层数`L=12`**，**隐藏层维度`H=768`**，**自注意力头数`A=12`**，**总参数量`110M`**

&emsp; 另外，模型参数自动下载至 home 目录下的`~\.cache\huggingface\transformers`文件夹中

In [ ]:
from fastNLP.transformers.torch import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
dir(tokenizer)

## 3. 实例：NG20 数据集的完整加载过程

### 3.1 使用 BertTokenizer 处理数据集

在`fastNLP 0.8`中，**`Trainer`模块和`Evaluator`模块分别表示“训练器”和“评测器”**

&emsp; 对应于之前的`fastNLP`版本中的`Trainer`模块和`Tester`模块，其定义方法如下所示

在`fastNLP 0.8`中，需要注意，在同个`python`脚本中先使用`Trainer`训练，然后使用`Evaluator`评测

&emsp; 非常关键的问题在于**如何正确设置二者的`driver`**。这就引入了另一个问题：什么是 `driver`？

In [ ]:
import pandas as pd

from fastNLP import DataSet
from fastNLP import Vocabulary

dataset = DataSet.from_pandas(pd.read_csv('./data/ng20_test.csv'))

In [ ]:
from functools import partial

encode = partial(tokenizer.encode_plus, max_length=100, truncation=True,
                 return_attention_mask=True)
# 会新增 input_ids 、 attention_mask 和 token_type_ids 这三个 field
dataset.apply_field_more(encode, field_name='text')

In [ ]:
target_vocab = Vocabulary(padding=None, unknown=None)

target_vocab.from_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='label')
target_vocab.index_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='label',
                           new_field_name='labels')
# 需要将 input_ids 的 pad 值设置为 tokenizer 的 pad 值
dataset.set_pad('input_ids', pad_val=tokenizer.pad_token_id)
dataset.set_ignore('label', 'text')  # 因为 label 是原始的不需要的 str ，所以我们可以忽略它，让它不要在 batch 的输出中出现